In [19]:
import pandas as pd
import requests
from tqdm import tqdm

In [2]:
url = './make_file/(KDT)_열선_취약시설_거리.csv'
file_encoding = 'EUC-KR'

In [3]:
data = pd.read_csv(url, encoding=file_encoding)

In [4]:
len(data)

569

In [5]:
data.columns

Index(['중점_위도', '중점_경도', '경사각', '최근접_시설의_평균거리', '열선여부'], dtype='object')

In [7]:
try :
    data = data.drop(columns="Unnamed: 0")
except :
    print('컬럼 없음')

컬럼 없음


## 행안부 API

In [9]:
api_key_url = '../API_KEY/API_KEYS.xlsx'
open_api_key_file = pd.read_excel(api_key_url)
 
open_api_key_file['사이트'].value_counts()

get_gov_api_key = open_api_key_file[open_api_key_file['사이트'] == '행정안전부'].values[0][1]  

In [11]:
# 변환할 지번 주소 입력 

def old_address_to_road_address(old_address) : 
    url = f"https://www.juso.go.kr/addrlink/addrLinkApi.do?confmKey={get_gov_api_key}&keyword={old_address}&resultType=json"
    
    # API 요청
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        # print(data)
        if "results" in data and "juso" in data["results"] and data["results"]["juso"]:
            road_addresses = [juso['roadAddr'] for juso in data['results']['juso']]
            return road_addresses[0]
        else:
            return None
    else:
        return None

## KAKAO API

In [12]:
api_key_url = '../API_KEY/API_KEYS.xlsx'
open_api_key_file = pd.read_excel(api_key_url)
 
open_api_key_file['사이트'].value_counts()

get_api_key = open_api_key_file[open_api_key_file['사이트'] == 'kakao_api_key_REST'].values[0][1]  

In [13]:
# KAKAO API KEY 값 
KAKAO_API_KEY = get_api_key  

In [21]:
# 서울시 광진구 위도, 경도 
def kakao_map_api(lat, lon):
    latitude = lat
    longitude = lon
    
    # 카카오 맵 API URL
    url = f"https://dapi.kakao.com/v2/local/geo/coord2address.json?x={longitude}&y={latitude}"
    
    # API 요청 헤더
    headers = {
        "Authorization": f"KakaoAK {KAKAO_API_KEY}"
    }
    
    # API 호출
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        
        # 데이터가 비어 있지 않으면 처리
        if data.get('documents'):
            region_3depth_name = data['documents'][0]['address'].get('region_3depth_name', '')
            temp_address_name = data['documents'][0]['address'].get('address_name', '')
            address_name = data['documents'][0].get('road_address', None)
            
            if address_name is None:
                temp = old_address_to_road_address(temp_address_name)
                if temp is None:
                    address_name = temp_address_name
                    is_road_address = 0  # False
                else:
                    address_name = temp
                    is_road_address = 1  # True
            else:
                is_road_address = 1  # True
                
            if isinstance(address_name, dict):
                address_name = address_name.get("address_name", "")  # 기본값을 빈 문자열로 설정
            
            # 문자열에서 특정 단어 제거
            address_name = address_name.replace("서울특별시 ", "")
            address_name = address_name.replace("서울 ", "") 
            
            return region_3depth_name, address_name, is_road_address
        else:
            return None, None, False  # 결과가 없을 경우 처리
    else:
        return None, None, False  # API 호출 실패 시 처리


In [22]:
lat_list = data["중점_위도"].to_list()
lon_list = data["중점_경도"].to_list()

print(len(lat_list))
print(len(lon_list))

569
569


In [23]:
dong_list = []
road_address_list = []
is_road_address_list = []

for i in tqdm(range(len(lat_list))) :
# for i in tqdm(range(60)) :
    lat = lat_list[i]
    lon = lon_list[i]
    dong, road_address, is_road_address = kakao_map_api(lat, lon)
    
    # print(dong, road_address, is_road_address)
    
    dong_list.append(dong)
    road_address_list.append(road_address)
    is_road_address_list.append(is_road_address)

100%|█████████████████████████████████████████| 569/569 [01:38<00:00,  5.78it/s]


In [24]:
print(len(dong_list))
print(len(road_address_list))
print(len(is_road_address_list))

569
569
569


In [25]:
data['행정동'] = dong_list
data['주소'] = road_address_list
data['도로명_여부'] = is_road_address_list

In [26]:
data["도로명_여부"].value_counts()

도로명_여부
1    418
0    151
Name: count, dtype: int64

In [33]:
data = data.sort_values('도로명_여부')
data = data.reset_index(drop=True)

In [34]:
try :
    save_file_url = './make_file/(최종)_열선_취약시설_거리.csv'
    save_file_encoding = 'EUC-KR'
    data.to_csv(save_file_url, encoding=save_file_encoding, index=False)
    print(f"{save_file_url} 저장 완료.")
except OSError as e :
    print(e)

./make_file/(최종)_열선_취약시설_거리.csv 저장 완료.
